# The Assignment of Week 3
### Import all the libraries I need in this notebook

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from bs4 import BeautifulSoup # library for web scrapting
from geopy.geocoders import Nominatim # import geocoder
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### web scrapting
   *Scrapt the table from wikipedia  
   *Assign three lists which will be used to form the dataframe

In [2]:
# assign the website address
web_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(web_url, 'lxml')

# prepare for scrapting
postalcodes, borough, neighborhood = [], [], [] #assign

#start to scrapting
for items in soup.find('table', class_ = 'wikitable sortable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        # appending data to the list
        postalcodes.append(data[0].text)
        borough.append(data[1].text)
        neighborhood.append(data[1].find_next_sibling().text.rstrip('\n'))
    except IndexError:pass

### Transfer Table to Dataframe
   *Collect three list to form the dataframe  
   *Drop all the rows have "Not Assigned" value

In [3]:
#Assign the list to sets with column name
df_dict = {'PostalCode': postalcodes, 'Borough': borough, 'Neighborhood': neighborhood}
#Dataframe
df = pd.DataFrame(data=df_dict)
#drop the not assigned value
df.drop(df[(df['Borough'] == 'Not assigned') | (df['Neighborhood'] == 'Not assigned')].index, inplace=True)
df = df.reset_index(drop=True).groupby(['PostalCode','Borough'], sort=False, as_index = False).agg( ','.join)

### Shape

In [4]:
df.shape

(102, 3)

With the dataframe I would assume that the neighborhood density could be displayed on the folium map. The density also is able to help us to acquire the information of the neighborhood from the foursquare

In [5]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M1B,Scarborough,"Rouge,Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens,Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M6B,North York,Glencairn


In [6]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
lat_lng_coords = pd.read_csv('Geospatial_Coordinates.csv')
lag_lng = pd.Series(map(lambda x: lat_lng_coords[['Latitude', 'Longitude']][lat_lng_coords['Postal Code'] == x].values.tolist()[0], df['PostalCode']))
lag_lng_df = pd.DataFrame(columns=['Latitude', 'Longitude'], data = list(lag_lng))
df = pd.concat([df, lag_lng_df], axis=1, sort=False)

In [7]:
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
6,M3B,North York,Don Mills North,43.745906,-79.352188
7,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
8,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073


In [8]:
#Get latitude and longityde of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postalcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df['PostalCode']):
    label = '{}, {} {}'.format(neighborhood, borough, postalcode) 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


In [9]:
def getNearbyVenues(names, latitudes, longitudes,CLIENT_ID, CLIENT_SECRET, VERSION,LIMIT, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
#Foursquare ID and Scecretc
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

df_dttor = df[df['Borough'].str.contains('Downtown Toronto')]
LIMIT = 100

dtToronto_venues = getNearbyVenues(names=df_dttor['Neighborhood'],
                                   latitudes=df_dttor['Latitude'],
                                   longitudes=df_dttor['Longitude'],
                                   CLIENT_ID=CLIENT_ID,
                                   CLIENT_SECRET=CLIENT_SECRET,
                                   VERSION=VERSION,
                                   LIMIT=LIMIT
                                  )
dtToronto_venues.to_csv('dtToronto_venues.csv')

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [11]:
dtToronto_venues=pd.read_csv('dtToronto_venues.csv').drop('Unnamed: 0', axis = 1)
dtToronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [12]:
dtToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",49,49,49,49,49,49
Central Bay Street,82,82,82,82,82,82
"Chinatown,Grange Park,Kensington Market",84,84,84,84,84,84
Christie,18,18,18,18,18,18
Church and Wellesley,83,83,83,83,83,83
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [13]:
print('There are {} uniques categories.'.format(len(dtToronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


### Analyze each neightborhood

In [14]:
# one hot encoding
dtToronto_onehot = pd.get_dummies(dtToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtToronto_onehot['Neighborhood'] = dtToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
Neighborhoodidx = list(dtToronto_onehot.columns).index('Neighborhood')
fixed_columns = [dtToronto_onehot.columns[Neighborhoodidx]] + list(dtToronto_onehot.columns[0:Neighborhoodidx]) + list(dtToronto_onehot.columns[Neighborhoodidx+1::])
dtToronto_onehot = dtToronto_onehot[fixed_columns]

In [15]:
dtToronto_onehot.shape

(1309, 206)

### Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category
### Create the top 10 most common venues of Dataframe by each neighborhood

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
dtToronto_grouped = dtToronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dtToronto_grouped['Neighborhood']

for ind in np.arange(dtToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Thai Restaurant,Bar,Café,Restaurant,Burger Joint,Bakery,Cosmetics Shop,Sushi Restaurant,Steakhouse
1,Berczy Park,Coffee Shop,Café,French Restaurant,Bakery,Steakhouse,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Park,Pub,Italian Restaurant,Pizza Place,Bakery,Office,Butcher
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Japanese Restaurant,Burger Joint,Ice Cream Shop,Thai Restaurant,Bakery,Café
5,"Chinatown,Grange Park,Kensington Market",Bar,Café,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Bakery,Vegetarian / Vegan Restaurant,Mexican Restaurant,Farmers Market
6,Christie,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Dance Studio,Hotel,Gastropub
8,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Gym,Italian Restaurant,Deli / Bodega,American Restaurant,Steakhouse,Seafood Restaurant
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Bar,Gastropub,Steakhouse


### Cluster Neighborhood
####    Due to the postal code catagories are only 19, the cluster is divided to 3 Clusters for analyze

In [18]:
# set number of clusters
kclusters = 3

dtToronto_grouped_clustering = dtToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtToronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtToronto_merged = df_dttor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtToronto_merged = dtToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dtToronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Restaurant,Mexican Restaurant,Dessert Shop
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Gym,Park,Yoga Studio,Juice Bar,Italian Restaurant,Portuguese Restaurant,Café,Bar,Fast Food Restaurant
8,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bakery,Ramen Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Breakfast Spot,Diner,Beer Bar,Bakery,American Restaurant
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,French Restaurant,Bakery,Steakhouse,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Japanese Restaurant,Burger Joint,Ice Cream Shop,Thai Restaurant,Bakery,Café
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store
29,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,0,Coffee Shop,Thai Restaurant,Bar,Café,Restaurant,Burger Joint,Bakery,Cosmetics Shop,Sushi Restaurant,Steakhouse
35,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Restaurant,Brewery,Scenic Lookout,Fried Chicken Joint,Plaza
41,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Bar,Gastropub,Steakhouse


### Visualize cluster map

In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtToronto_merged['Latitude'], dtToronto_merged['Longitude'], dtToronto_merged['Neighborhood'], dtToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Showing 5 Clusters

#### First Cluster 

Based on the top 10 most common venues in the first cluster, I assume that the neighborhoods are the place people visit for entertainment. Therefore, I will name the cluster to entertainment

In [20]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 0, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Restaurant,Mexican Restaurant,Dessert Shop
4,Downtown Toronto,0,Coffee Shop,Gym,Park,Yoga Studio,Juice Bar,Italian Restaurant,Portuguese Restaurant,Café,Bar,Fast Food Restaurant
8,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bakery,Ramen Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place
14,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Breakfast Spot,Diner,Beer Bar,Bakery,American Restaurant
19,Downtown Toronto,0,Coffee Shop,Café,French Restaurant,Bakery,Steakhouse,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop
23,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Japanese Restaurant,Burger Joint,Ice Cream Shop,Thai Restaurant,Bakery,Café
24,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store
29,Downtown Toronto,0,Coffee Shop,Thai Restaurant,Bar,Café,Restaurant,Burger Joint,Bakery,Cosmetics Shop,Sushi Restaurant,Steakhouse
35,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Restaurant,Brewery,Scenic Lookout,Fried Chicken Joint,Plaza
41,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Bar,Gastropub,Steakhouse


#### Second Cluster

Based on the top 10 most common venues in the Second cluster, I assume that the neighborhood is the place for living; park, playground, trail, and most of the venues are normally surround with the area for living. Therefore, I will name the cluster to residential area.

In [21]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 1, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Third Cluster

Based on the top 10 most common venues in the Third cluster, I assume that the neighborhood is near to the airport. Therefore, I will name the cluster to airport

In [22]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 2, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry


### Clusters label rename

In [23]:
dtToronto_merged = dtToronto_merged.replace(0,'Entertainment').replace(1,'Residential area').replace(2, 'Airport')
dtToronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,Entertainment,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Restaurant,Mexican Restaurant,Dessert Shop
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,Entertainment,Coffee Shop,Gym,Park,Yoga Studio,Juice Bar,Italian Restaurant,Portuguese Restaurant,Café,Bar,Fast Food Restaurant
8,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,Entertainment,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bakery,Ramen Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Entertainment,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Breakfast Spot,Diner,Beer Bar,Bakery,American Restaurant
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Entertainment,Coffee Shop,Café,French Restaurant,Bakery,Steakhouse,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Entertainment,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Japanese Restaurant,Burger Joint,Ice Cream Shop,Thai Restaurant,Bakery,Café
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Entertainment,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store
29,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,Entertainment,Coffee Shop,Thai Restaurant,Bar,Café,Restaurant,Burger Joint,Bakery,Cosmetics Shop,Sushi Restaurant,Steakhouse
35,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,Entertainment,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Restaurant,Brewery,Scenic Lookout,Fried Chicken Joint,Plaza
41,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,Entertainment,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Bar,Gastropub,Steakhouse
